In [1]:
import pandas as pd
import numpy as np
from hatchet import *
from state import State
import os
import json
from abc import ABCMeta, abstractmethod

In [2]:
name = 'kripke-impi'
dir_name = "/home/vidi/Work/llnl/CallFlow/data/kripke/"

In [3]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df


In [4]:
def read_gf(name):
        state = State()
        path = dir_name + 'impi'
        df_filepath = path + '/df.csv'
        entire_df_filepath = path + '/entire_df.csv'
        graph_filepath = path + '/graph.json'
        entire_graph_filepath = path + '/entire_graph.json'   

        with open(graph_filepath, 'r') as graphFile:
            data = json.load(graphFile)

        state.gf = GraphFrame()
        state.gf.from_literal(data)

        with open(entire_graph_filepath, 'r') as entire_graphFile:
            entire_data = json.load(entire_graphFile)
            
        state.entire_gf = GraphFrame()
        state.entire_gf.from_literal(entire_data)

        state.df = pd.read_csv(df_filepath)
        state.entire_df = pd.read_csv(entire_df_filepath)

        state.graph = state.gf.graph
        state.entire_graph = state.entire_gf.graph

        state.map = state.node_hash_mapper()

        # Print the module group by information. 
        # print(state.df.groupby(['module']).agg(['mean','count']))

        # replace df['node'] from str to the Node object.
        state.df = replace_str_with_Node(state.df, state.graph)
        state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)

        # add path to the dataframes. 
        # state.df['path'] = state.df['node'].apply(lambda node: node.callpath)
        # state.entire_df['path'] = state.entire_df['node'].apply(lambda node: node.callpath if node else [])

        return state

In [5]:
states = {}
states[name] = read_gf(name)

In [6]:
class Action():
    __metaclass__ = ABCMeta
     
    def __init__(self):
        pass
    
    def dfs(self, graph, limit):
        self.level = 0
        
        def dfs_recurse(root):
            for node in root.children:
                if(self.level < limit):
                    print("Node : ", node)
                    self.level += 1
                    dfs_recurse(node)
        
        for root in graph.roots:
            dfs_recurse(root)

    def create_group_path(self, node):
        group_path = []
        temp = None
        path = node.callpath
        for i, elem in enumerate(path):
            elem_df = self.state.lookup_with_nodeName(elem)
            if(elem_df.empty):
                grouping = 'Unkno'
            else:
                grouping = elem_df[self.group_by].tolist()[0]
            if temp == None or grouping != temp:
                group_path.append(grouping)
                temp = grouping
        return tuple(group_path)

    def find_a_good_node_name(self, node):
        node_name = self.state.lookup_with_node(node)[self.group_by].tolist()[0]
        return node_name

    def create_component_path(self, path, group_path):
        component_path = []
        path = list(path)
        component_module = self.state.lookup_with_nodeName(path[-1])[self.group_by].tolist()[0]
        component_path.append(component_module)

        filter_path = [node for node in path if component_module == self.state.lookup_with_nodeName(node)[self.group_by].tolist()[0]]
       
        for i, elem in enumerate(filter_path):            
             component_path.append(elem)                    
        return tuple(component_path)

    def create_component_level(self, component_path):
        return len(component_path) - 1

    # Assign a "Module" name to a given hatchet node. 
    def give_module_name(self, node):
        # start = time.time()
        df = self.state.lookup_with_nodeName(node.callpath[-1])
        unique_modules = df[self.group_by].unique()
        if(len(unique_modules) == 1):
            return unique_modules[0]
        elif(len(unique_modules) == 0): 
            # print('Entry not found in dataframe')
            return 'Unkno'
        else:
            print('Error! Multiple modules for a node.')
            return None

    @abstractmethod
    def run():
        pass

In [7]:
class groupGraph(Graph):
    """ A group node in the call graph.
    """
    def __init__(self):
        self.nodes = []
        self.edges = []
        # Map to check if such element exist in the graph. 
        self.nodeMap = {}
        self.roots = []
    
    def is_module(self, node_name):
        for idx, module in enumerate(self.nodes):
            if(node_name == module.callpath[-1]):
                return True
        return False

    def add_module(self, module):
        assert isinstance(module, Node)
        self.nodes.append(module)
        self.nodeMap[module.callpath[-1]] = module

    def add_inner_node(self, module_name, inner_node):
        assert isinstance(inner_node, Node)
        group_node = self.nodeMap[module_name]
        self.nodeMap[module_name] = inner_node
        group_node.hierarchy.append(inner_node)

    def print(self):
        print("Nodes: ", self.nodes)
        for idx, node in enumerate(self.nodes):
            print("Node name: {0}, \n hierarchy: {1}".format(node.name, node.hierarchy))
    def split_by_entry_function(self):
        return 

    def split_by_caller_function(self):
        return

    def get_hierarchy():
        return

    def split_level(self):
        return 

In [9]:
class groupNode(Node):
    def __init__(self, nid, name, callpath_tuple, source):
        self.nid = nid
        self.name = name
        self.callpath = callpath_tuple
        self.parents = []
        
        if source is not None:
            self.add_parent(source)
        self.children = []
        self.entry_funcs = []
        self.caller_funcs = []
        self.hierarchy = []

    def add_parent(self, node):
        self.parents.append(node)

    def add_child(self, node):
        assert isinstance(node, groupNode)
        self.children.append(node)

    def add_entry_funcs(self, node):
        assert isinstance(node, Node)
        self.entry_funcs.append(node)

    def __str__(self):
        """ Returns a string representation of the node.
        """
        return '[Node] name: {0}, number_of_parents: {1}, number_of_children: {2}, entry_funcs: {3}'.format(self.callpath, len(self.parents), len(self.children), len(self.entry_funcs))


In [70]:
class groupBy(Action):
    def __init__(self, state, group_by):
        self.state = state
        self.graph = state.graph
        self.df = state.df
        self.entire_df = state.entire_df
        self.group_by = group_by
        self.node_count = 0
        self.empty_node = Node(-1, (), None)
        self.nodeModuleMap = self.find_mapping_from_df()
        print(self.nodeModuleMap.keys())
        self.run()

    def find_mapping_from_df(self):
        fns = self.df['name'].unique()
        nodeModuleMap = {}
        for idx, fn in enumerate(fns):
            fn_hatchet = self.df[self.df['name'] == fn]['node'].unique()[0]
            nodeModuleMap[fn] = self.df[self.df['name'] == fn]['module'].unique()
        return nodeModuleMap
        
    def add_node(self, source, target, is_root=False):
        count = self.node_count
        
        target_name = target.callpath[-1]
        target_module_name = self.give_module_name(target)
        target_callpath = self.create_group_path(target)
        if (source == self.empty_node):
            pass
        else:
            source_name = source.callpath[-1]
            source_module_name = self.give_module_name(source)
            source_callpath = self.create_group_path(source)

            print(source_callpath, target_callpath)
        # hatchet_hash = self.state.lookup_with_node(node.callpath[-1])['node'].unique()

 
        if(is_root):
            source_module_name = None
            source_callpath = None
            group_node = groupNode(count, target_module_name, source_callpath, source_module_name)
            self.group_graph.roots.append(group_node)
            
        else:
            if(self.group_graph.is_module(target_module_name)):
                self.group_graph.add_inner_node(target_module_name, target)
            else:
                if(len(self.group_graph.nodes) == 0):
                    source_module = None
                else:
                    source_module = self.nodeModuleMap[source_name]
                    group_node = groupNode(count, target_module_name, target_callpath, source_module_name)
            self.group_graph.add_module(target)
        
    def run(self):         
        roots = self.graph.roots
        # New roots of the grouped graph.
        new_roots = []
        self.group_graph = groupGraph()
        
        for root in roots:
            level = 0
            node_gen = root.traverse()  
            self.add_node(self.empty_node, root, True)
#             print(self.group_graph.nodeMap)

            try:
                while root.callpath != None:
                    source = root
                    target = next(node_gen)
                    self.add_node(source, target, False)
                    # self.add_edge(target, source)
                    root = target

            except StopIteration:
                pass
            finally:
                del root

        self.group_graph.print()

        group_roots = self.group_graph.roots
        print(self.group_graph.nodeMap)
#         for root in group_roots:
#             group_node_gen = root.traverse()
#             print(root.callpath)
#             try:
#                 while root.callpath != None:
#                     source = root
#                     target = next(group_node_gen)
#                     print(source, target)
#             except StopIteration:
#                 pass
#             finally: 
#                 del root
           

In [71]:
groupBy(states[name], 'module')

dict_keys(['<program root>', 'main', 'SweepSolver', 'Loop@<unknown file> [kripke]:0', 'Grid_Data::particleEdit', 'MPIDI_SHM_mpi_reduce', '247:MPIDI_SHMGR_Release', '900:MPIDI_SHMGR_release_generic', '<unknown file> [libmpi.so.12.0.0]:329', 'Kernel_3d_DGZ::LPlusTimes', '<unknown file> [kripke]:0', 'Kernel_3d_DGZ::LTimes', '__memset_sse2', 'Kernel_3d_DGZ::scattering', 'SweepSubdomains', 'Kernel_3d_DGZ::sweep', 'SweepComm::readySubdomains', 'ParallelComm::testRecieves', 'MPID_Progress_test'])
('libmonitor.so.0.0.0',) ('libmonitor.so.0.0.0',)
('libmonitor.so.0.0.0',) ('libmonitor.so.0.0.0', 'kripke')
('libmonitor.so.0.0.0', 'kripke') ('libmonitor.so.0.0.0', 'kripke')
('libmonitor.so.0.0.0', 'kripke') ('libmonitor.so.0.0.0', 'kripke', 'Unkno')
('libmonitor.so.0.0.0', 'kripke', 'Unkno') ('libmonitor.so.0.0.0', 'kripke', 'Unkno', 'kripke')
('libmonitor.so.0.0.0', 'kripke', 'Unkno', 'kripke') ('libmonitor.so.0.0.0', 'kripke', 'Unkno', 'kripke', 'Unkno')
('libmonitor.so.0.0.0', 'kripke', 'Unkno

KeyError: 'memset.S:98'

In [17]:
state = states[name]
print(state.df[state.df['name'] == "Loop@<unknown file> [kripke]:0"])

                                              node  rank   time (inc)  time  \
384   <hatchet.node.Node object at 0x7f13d34d8e80>     0  193933103.0   0.0   
385   <hatchet.node.Node object at 0x7f13d34d8e80>     1  193941625.0   0.0   
386   <hatchet.node.Node object at 0x7f13d34d8e80>     2  193981015.0   0.0   
387   <hatchet.node.Node object at 0x7f13d34d8e80>     3  194023154.0   0.0   
388   <hatchet.node.Node object at 0x7f13d34d8e80>     4  193957846.0   0.0   
389   <hatchet.node.Node object at 0x7f13d34d8e80>     5  193955999.0   0.0   
390   <hatchet.node.Node object at 0x7f13d34d8e80>     6  193994342.0   0.0   
391   <hatchet.node.Node object at 0x7f13d34d8e80>     7  193916176.0   0.0   
392   <hatchet.node.Node object at 0x7f13d34d8e80>     8  193881701.0   0.0   
393   <hatchet.node.Node object at 0x7f13d34d8e80>     9  193881630.0   0.0   
394   <hatchet.node.Node object at 0x7f13d34d8e80>    10  194041468.0   0.0   
395   <hatchet.node.Node object at 0x7f13d34d8e80>  

In [35]:
state = states[name]
fns = state.entire_df['name'].unique()
nodeModuleMap = {}
for idx, fn in enumerate(fns):
    fn_hatchet = state.entire_df[state.entire_df['name'] == fn]['node'].unique()[0]
    nodeModuleMap[fn_hatchet] = state.df[state.df['name'] == fn]['module'].unique()
print(nodeModuleMap)

{<hatchet.node.Node object at 0x7f34a57eb320>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a57eb438>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a57ebc18>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a57eb2e8>: array(['libmonitor.so.0.0.0'], dtype=object), <hatchet.node.Node object at 0x7f34a57eb828>: array(['kripke'], dtype=object), <hatchet.node.Node object at 0x7f34a57eb550>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a5913080>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a59130f0>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a5913198>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a5913208>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a5913278>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a59132e8>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a5913358>: array([], dtype=object), <hatchet.node.Node object at 0x7f34a59133c8>: a